<div align='center'><font size="5" color='#353B47'>Valorant BETA</font></div>
<div align='center'><font size="4" color="#353B47">Will they dethrone Counter Strike ?</font></div>
<br>
<div align='center'><img src = "https://www.kolpaper.com/wp-content/uploads/2020/04/valorant-wallpaper-4k-scaled.jpg"></div>
<br>
<hr>

<div align="justify"><font size='3' color ='#FF4654'>Here is a quick notebook to parse the tweets.txt file into a regular pandas dataframe. The data were retrieved using Twitter API and stored into a JSON file. The second part of this notebook will perform a quick EDA on the data. The EDA can be generalized to any tweets dataframe with the same structure. I hope you'll enjoy !</font></div>

# <font color ='#261E26'>1. Libraries</font>

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd
from textblob import TextBlob
from pandas.io.json import json_normalize
from wordcloud import WordCloud
import math
import re
import json

In [ ]:
# Preprocessing functions

def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
def analyze_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1
    
def convert_month_to_number(month):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return months.index(month)+1

def set_timestamp(df):
    dates = pd.DataFrame(columns = ['Year', 'Month', 'Day'])
    dates['Year'] = df['Date'].apply(lambda x: int(x[-4:]))
    dates['Month'] = df['Date'].apply(lambda x: x[4:7]).apply(lambda x:int(convert_month_to_number(x)))
    dates['Day'] = df['Date'].apply(lambda x: int(x[8:10].rsplit()[0]))

    df.Date = pd.to_datetime(dates)
    return df

# <font color ='#261E26'>2. From JSON to Pandas Dataframe</font>

In [ ]:
column_names = ['Date', 'Text', 'Likes', 'Retweets', 'Sentiment', 'Source', 'Length']

# Create empty dataframe with column names
valorant_df = pd.DataFrame(columns = column_names)

with open('../input/valorant-tweets/tweets.txt') as f:
    for line in f:
        if len(line) > 10: # Dont take into account empty lines
            to_append = {}
            obs = json.loads(line)
            
            to_append['Date'] = obs['created_at']
            to_append['Text'] = obs['text']
            to_append['Likes'] = obs['favorite_count'] 
            to_append['Retweets'] = obs['retweet_count']
            
            valorant_df = valorant_df.append(to_append, ignore_index = True)

valorant_df = set_timestamp(valorant_df)
valorant_df['Sentiment'] = valorant_df['Text'].apply(lambda x:analyze_sentiment(x))
valorant_df['Source'] = 'random_user'
valorant_df['Length'] = valorant_df['Text'].apply(len)
valorant_df['Word_counts'] = valorant_df['Text'].apply(lambda x:len(str(x).split()))

In [ ]:
valorant_df.head()

In [ ]:
print(f'{valorant_df.shape[0]} observations, {valorant_df.shape[1]} columns')

# <font color ='#261E26'>3. Cleaning tweets</font>

In [ ]:
# Find emoji patterns
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

# Basic function to clean the text
def clean_text(text):
    text = str(text)
    # Remove emojis
    text = emoji_pattern.sub(r'', text)
    # Remove identifications
    text = re.sub(r'@\w+', '', text)
    # Remove links
    text = re.sub(r'http.?://[^/s]+[/s]?', '', text)
    return text.strip().lower()

In [ ]:
valorant_df['Text'] = valorant_df['Text'].apply(lambda x:clean_text(x))

# <font color ='#261E26'>4. Quick EDA</font>

### <font color ='#FF4654'>Distribution of sentiment</font>

In [ ]:
# Count of observations per category
valorant_df['Sentiment'].value_counts(normalize = True)

In [ ]:
x = ['Neutral', 'Positive', 'Negative']
y = [6959, 3177, 1338]

# Use the hovertext kw argument for hover text
fig = go.Figure(data=[go.Bar(x=x, y=y,
            hovertext=['61% of tweets', '28% of tweets', '11% of tweets'])])

# Customize aspect
#marker_color='rgb(158,202,225)'
fig.update_traces(marker_line_color='midnightblue',
                  marker_line_width=1.)
fig.update_layout(title_text='Distribution of sentiment')
fig.show()

### <font color ='#FF4654'>Examples of sentiment</font>

In [ ]:
neutral = valorant_df[valorant_df['Sentiment'] == 0]
positive = valorant_df[valorant_df['Sentiment'] == 1]
negative = valorant_df[valorant_df['Sentiment'] == -1]

In [ ]:
#neutral_text
print("Neutral tweet example  :",neutral['Text'].values[1])
# Positive tweet
print("Positive Tweet example :",positive['Text'].values[1])
#negative_text
print("Negative Tweet example :",negative['Text'].values[1])

### <font color ='#FF4654'>Distribution of tweet lengths</font>

In [ ]:
x = valorant_df.Length.values
#x = [math.log10(i) for i in list(valorant_df.Length.values) if i!= 0]

fig = go.Figure(data=[go.Histogram(x=x,
                                   marker_line_width=1, 
                                   marker_line_color="midnightblue", 
                                   xbins_size = 5)])

fig.update_layout(title_text='Distribution of tweet lengths')
fig.show()

In [ ]:
x1 = neutral.Length.values
x2 = positive.Length.values
x3 = negative.Length.values

fig = go.Figure(data=[go.Histogram(x=x1,
                                   marker_line_width=1, 
                                   marker_line_color="midnightblue", 
                                   xbins_size = 5,
                                   opacity = 0.5)])

fig.update_layout(title_text='Distribution of neutral tweet lengths')
fig.show()

fig = go.Figure(data=[go.Histogram(x=x2,
                                   marker_line_width=1, 
                                   marker_color='rgb(50,202,50)', 
                                   marker_line_color="midnightblue", 
                                   xbins_size = 5,
                                   opacity = 0.5)])

fig.update_layout(title_text='Distribution of positive tweet lengths')
fig.show()

fig = go.Figure(data=[go.Histogram(x=x3,
                                   marker_line_width=1, 
                                   marker_color='crimson', 
                                   marker_line_color="midnightblue", 
                                   opacity = 0.5)])

fig.update_layout(title_text='Distribution of negative tweet lengths')
fig.show()

In [ ]:
y1 = neutral.Length.values
y2 = positive.Length.values
y3 = negative.Length.values

fig = go.Figure()

fig.add_trace(go.Box(y=y1, 
                     name="Neutral", 
                     marker_line_width=1, 
                     marker_line_color="midnightblue"))

fig.add_trace(go.Box(y=y2, 
                     name="Positive", 
                     marker_line_width=1, 
                     marker_color = 'rgb(50,202,50)'))

fig.add_trace(go.Box(y=y3, 
                     name="Negative", 
                     marker_line_width=1, 
                     marker_color = 'crimson'))

fig.update_layout(title_text="Box Plot tweet lengths")

fig.show()

### <font color ='#FF4654'>Distribution of word counts</font>

In [ ]:
x = valorant_df.Word_counts.values
#x = [math.log10(i) for i in list(valorant_df.Length.values) if i!= 0]

fig = go.Figure(data=[go.Histogram(x=x,
                                   marker_line_width=1, 
                                   marker_line_color="midnightblue")])

fig.update_layout(title_text='Distribution of tweet lengths')
fig.show()

In [ ]:
x1 = neutral.Word_counts.values
x2 = positive.Word_counts.values
x3 = negative.Word_counts.values

fig = go.Figure(data=[go.Histogram(x=x1,
                                   marker_line_width=1, 
                                   marker_line_color="midnightblue", 
                                   opacity = 0.5)])

fig.update_layout(title_text='Distribution of neutral tweet lengths')
fig.show()

fig = go.Figure(data=[go.Histogram(x=x2,
                                   marker_line_width=1, 
                                   marker_color='rgb(50,202,50)', 
                                   marker_line_color="midnightblue", 
                                   opacity = 0.5)])

fig.update_layout(title_text='Distribution of positive tweet lengths')
fig.show()

fig = go.Figure(data=[go.Histogram(x=x3,
                                   marker_line_width=1, 
                                   marker_color='crimson', 
                                   marker_line_color="midnightblue", 
                                   opacity = 0.5)])

fig.update_layout(title_text='Distribution of negative tweet lengths')
fig.show()

In [ ]:
y1 = neutral.Word_counts.values
y2 = positive.Word_counts.values
y3 = negative.Word_counts.values

fig = go.Figure()

fig.add_trace(go.Box(y=y1, 
                     name="Neutral", 
                     marker_line_width=1, 
                     marker_line_color="midnightblue"))

fig.add_trace(go.Box(y=y2, 
                     name="Positive", 
                     marker_line_width=1, 
                     marker_color = 'rgb(50,202,50)'))

fig.add_trace(go.Box(y=y3, 
                     name="Negative", 
                     marker_line_width=1, 
                     marker_color = 'crimson'))

fig.update_layout(title_text="Box Plot word counts")

fig.show()

# <font color ='#261E26'>5. Wordclouds</font>

In [ ]:
def wordcloud(df, text = 'Text'):
    
    # Join all tweets in one string
    corpus = " ".join(str(review) for review in df[text])
    
    print (f"There are {len(corpus)} words in the combination of all review.")
    
    wordcloud = WordCloud(max_font_size=50, 
                          max_words=100, 
                          collocations=False,
                          background_color="white").generate(corpus)
    
    plt.figure(figsize=(15,15))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
print('Neutral Wordcloud')
wordcloud(df = neutral)

print('Positive Wordcloud')
wordcloud(df = positive)

print('Negative Wordcloud')
wordcloud(df = negative)

* <font size = '3' color ='#FF4654'>Negative words in this wordcloud like bored, bad, suck...</font>
* <font size = '3' color ='#0F6105'>Positive words: good, great, thank, nice...</font>

-----

# <font color ='#261E26'>6. Final Dataframe</font>

### <font color ='#FF4654'>Merging tweets with tweets posted by community managers</font>

In [ ]:
# This dataset contains all tweets from official Valorant accounts
from_RIOT = pd.read_csv('../input/valorant-tweets/Valorant.csv', usecols = ['Date', 'Text', 'Likes', 'Retweets', 'Sentiment', 'Source', 'Length'])
from_RIOT = from_RIOT.reindex(columns=['Date', 'Text', 'Likes', 'Retweets', 'Sentiment', 'Source', 'Length'])
from_RIOT['Date'] = pd.to_datetime(from_RIOT['Date'], format ='%Y/%m/%d')
from_RIOT['Word_counts'] = from_RIOT['Text'].apply(lambda x:len(str(x).split()))

In [ ]:
# Merge the dataframes is possible has we have the source of accounts each tweet comes from
dataframe = pd.concat([valorant_df, from_RIOT])
dataframe.to_csv('all_tweets.csv')

<div align="center"><img src = "https://images6.alphacoders.com/107/thumb-1920-1072679.jpg"></div>

<hr>
<br>
<div align='justify'><font color="#353B47" size="4">Thank you for taking the time to read this notebook. I hope that I was able to answer your questions or your curiosity and that it was quite understandable. <u>any constructive comments are welcome</u>. They help me progress and motivate me to share better quality content. I am above all a passionate person who tries to advance my knowledge but also that of others. If you liked it, feel free to <u>upvote and share my work.</u> </font></div>
<br>
<div align='center'><font color="#353B47" size="3">Thank you and may passion guide you.</font></div>